# Bayesian Hierarichcal Models:

## Centered vs Non-centered models

This notebook discusses the differences between centered and non-centered models.

It also highlights how a non-centered model can successfully explore certain areas of the posterior that a centered model will miss.

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm

## A (centered) model

Consider the following model:

There are $I$ groups within our data. For each group ($i$), data is generated from:

$$y_{i, n} = \beta_i x_{i, n} + \varepsilon_{i, n}$$

where $\varepsilon_{i, n} \sim N(0, \sigma)$.

### Priors

Suppose that we believe that each $\beta_i$ is drawn from

\begin{align*}
  \beta_i &\sim N(\mu_\beta, \sigma_\beta) \\
\end{align*}

where we believe that

\begin{align*}
  \mu_\beta &\sim N(0, 5) \\
  \sigma_\beta &\sim \text{HalfCauchy}(3)
\end{align*}

and $\sigma$ is drawn from

\begin{align*}
  \sigma &\sim \text{HalfCauchy}(3) \\
\end{align*}


In [ ]:
# Parameters
mu_beta_mean = 0.0
mu_beta_std = 5.0
sigma_beta_param = 3.0
sigma_param = 3.0
I = 35
nobs = 7

# Data
np.random.seed(1234)
dgp_mu_beta = 1.0
dgp_sigma_beta = 0.25
dgp_sigma = 0.75
dgp_betas = dgp_mu_beta + dgp_sigma_beta*np.random.randn(I)

idxs = np.tile(np.arange(I), nobs)
x = np.random.randn(I*nobs)
y = dgp_betas[idxs]*x + dgp_sigma*np.random.randn(I*nobs)

centered_m = pm.Model()

In [ ]:
with centered_m:
    # Data
    data_x = pm.Data("x", x)
    data_idx = pm.Data("idx", idxs)
    data_y = pm.Data("y", y)

    # Hyperpriors
    mu_beta = pm.Normal("mu_beta", mu_beta_mean, mu_beta_std)
    sigma_beta = pm.HalfCauchy("sigma_beta", sigma_beta_param)

    # Priors
    beta = pm.Normal("betas", mu_beta, sigma_beta, shape=(I,))
    sigma = pm.HalfCauchy("sigma", sigma_param)

    # Likelihood
    obs = pm.Normal("likelihood", beta[data_idx]*data_x, sigma, observed=data_y)

### Sample from posterior



In [ ]:
with centered_m:
    centered_traces = pm.sample(
        1_500, tune=1_000, target_accept=0.8,
        return_inferencedata=False
    )

In [ ]:
with centered_m:
    az.plot_trace(centered_traces)

    plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(14, 16))

centered_sigma_betas = centered_traces.get_values("sigma_beta", combine=False)
for i in range(4):
    ax[i, 0].hist(centered_sigma_betas[i], bins=50, density=True)
    ax[i, 0].set_xlim(0.0, 0.75)

    ax[i, 1].plot(centered_sigma_betas[i])
    ax[i, 1].set_ylim(0.0, 0.75)

## Non-centered model

In [ ]:
noncentered_m = pm.Model()

with noncentered_m:
    # Data
    data_x = pm.Data("x", x)
    data_idx = pm.Data("idx", idxs)
    data_y = pm.Data("y", y)

    # Hyperpriors
    mu_beta = pm.Normal("mu_beta", mu_beta_mean, mu_beta_std)
    sigma_beta = pm.HalfStudentT("sigma_beta", sigma_beta_param)

    # Priors
    b_offset = pm.Normal("b_offset", 0.0, 1.0, shape=(I,))
    beta = pm.Deterministic("betas", mu_beta + sigma_beta*b_offset)
    sigma = pm.HalfStudentT("sigma", sigma_param)

    # Likelihood
    obs = pm.Normal("likelihood", beta[data_idx]*data_x, sigma, observed=data_y)

Modified excerpt from https://twiecki.io/blog/2017/02/08/bayesian-hierchical-non-centered/

> Pay attention to the definitions of `beta` and `b_offset`... What's going on here? It's pretty neat actually. Instead of saying that our individual slopes `beta` are normally distributed around a group mean (i.e. modeling their absolute values directly), we can say that they are offset from a group mean by a certain value (`b_offset`; i.e. modeling their values relative to that mean). Now we still have to consider how far from that mean we actually allow things to deviate (i.e. how much shrinkage we apply). This is where `sigma_b` makes a comeback. We can simply multiply the offset by this scaling factor to get the same effect as before, just under a different parameterization.

### Sample from posterior


In [ ]:
with noncentered_m:
    noncentered_traces = pm.sample(
        1_500, tune=1_000, target_accept=0.8,
        return_inferencedata=False
    )

In [ ]:
with noncentered_m:

    az.plot_trace(noncentered_traces)
    plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(14, 16))

noncentered_sigma_betas = noncentered_traces.get_values("sigma_beta", combine=False)
for i in range(4):
    ax[i, 0].hist(noncentered_sigma_betas[i], bins=50, density=True)
    ax[i, 0].set_xlim(0.0, 0.75)

    ax[i, 1].plot(noncentered_sigma_betas[i])
    ax[i, 1].set_ylim(0.0, 0.75)

## Are these actually any different?

**Differences in $\sigma_\beta$**

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(8, 12))

# Data from centered
c_sigma_beta = centered_traces["sigma_beta"][:]

# Data from non-centered
nc_sigma_beta = noncentered_traces["sigma_beta"][:]

ax[0].hist(c_sigma_beta, bins=50, density=True)
ax[0].set_xlim(0, 0.9)

ax[1].hist(nc_sigma_beta, bins=50, density=True)
ax[1].set_xlim(0, 0.9)

What fraction of draws of $\sigma_\beta$ are less than 0.1 in each model?

In [ ]:
np.mean(c_sigma_beta < 0.10)

In [ ]:
np.mean(nc_sigma_beta < 0.10)

**Differences in $\beta$**

In [ ]:
np.argmin(dgp_betas)

In [ ]:
np.argmax(dgp_betas)

In [ ]:
i = 2

# Data from centered
c_beta_i = centered_traces["betas"][:, i]

# Data from non-centered
nc_beta_i = noncentered_traces["betas"][:, i]

fig, ax = plt.subplots(1, 2, figsize=(12, 8))

ax[0].hist(c_beta_i, bins=50, density=True)
ax[0].vlines(dgp_betas[i], 0.0, 5.0, color="k", linewidth=2.0)
ax[0].set_xlim(0.0, 2.5)
ax[0].set_ylim(0.0, 2.5)

ax[1].hist(nc_beta_i, bins=50, density=True)
ax[1].vlines(dgp_betas[i], 0.0, 5.0, color="k", linewidth=2.0)
ax[1].set_xlim(0.0, 2.5)
ax[1].set_ylim(0.0, 2.5)

## Pair plot comparison

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 12))

i = 1

# Data from centered
c_beta_0 = centered_traces["betas"][:, i]
c_sigma_beta = centered_traces["sigma_beta"][:]

# Data from non-centered
nc_beta_0 = noncentered_traces["betas"][:, i]
nc_sigma_beta = noncentered_traces["sigma_beta"][:]

ax[0].scatter(c_beta_0, c_sigma_beta)
ax[0].set_ylim(0.0, 0.75)

ax[1].scatter(nc_beta_0, nc_sigma_beta)
ax[1].set_ylim(0.0, 0.75)